In [1]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_transmitter_task import SingleChTransmitterTask
from rfsoc_rfdc.receiver.single_ch_receiver_task import SingleChReceiverTask

from rfsoc_rfdc.transmitter.multi_ch_transmitter_task import MultiChTransmitterTask
from rfsoc_rfdc.receiver.multi_ch_receiver_task import MultiChReceiverTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

In [ ]:
# RFSoC experiment configuration
# +--------------------------------------------------------------------------------------+
# | Feature: Single channel 100MHz bandwidth (Tx only)                                   |
# +-----------------+--------------------+----------------+-----------+-----+------------+
# | DAC Sample Rate | Interpolation Rate | IQ Sample Rate | Bandwidth | NCO | Ref. Clock |
# | 2.0 GSPS        | 20x                | 200 MSPS       | 100M      | +1G | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-----+------------+
# | ADC Sample Rate | Decimation Rate    | IQ Sample Rate | Bandwidth | NCO | Ref. Clock |
# | 2.0 GSPS        | 20x                | 200 MSPS       | 100M      | -1G | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-----+------------+

In [ ]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v27.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 2000.0,
    "DACInterpolationRate": 2,
    "DACNCO": 1000.0,
    "ADCSampleRate": 2000.0,
    "ADCInterpolationRate": 2,
    "ADCNCO": -1000.0
}
ZCU216_CONFIG.update(NEW_CONFIG)

In [ ]:
import sys
import os

rfdc_t = RfdcTask(ol)
led_t = BlinkLedTask(ol)
tx_t = SingleChTransmitterTask(ol, "./wave_files/Tx.mat")
rx_t = SingleChReceiverTask(ol)

for task in [rfdc_t]:
    task.start()
    task.join()

parallel_task = [led_t, tx_t]
for task in parallel_task:
    task.start()
for task in parallel_task:
    task.join()

axi_gpio_led
axi_rfdc_intc
usp_rf_data_converter
dac_datapath/t230/axi_dma
dac_datapath/t230/fifo_count
adc_datapath/t226/adc_packet_generator
adc_datapath/t226/axi_dma
adc_datapath/t226/fifo_count
zynq_ultra_ps_e


FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'title': {'text': 'Complex Sig…

FigureWidget({
    'data': [{'name': 'Magnitude', 'type': 'scattergl', 'uid': '72515a9c-8c1a-466e-bbc5-ddf4cc3…

2024-05-15 03:57:15,270 - root - INFO - Configuring LMK chip using file LMK04828_250.0.txt at 250.0 MHz
2024-05-15 03:57:15,276 - root - INFO - Configuring LMX chip using file LMX2594_500.0.txt at 500.0 MHz
2024-05-15 03:57:17,736 - root - INFO - DAC tile 1 (229) is fully powered up!
2024-05-15 03:57:18,785 - root - INFO - DAC tile 2 (230) is fully powered up!
2024-05-15 03:57:18,787 - root - INFO - DAC tile 0 DAC block 0 is NOT enabled!
2024-05-15 03:57:18,790 - root - INFO - DAC tile 0 DAC block 1 is NOT enabled!
2024-05-15 03:57:18,792 - root - INFO - DAC tile 0 DAC block 2 is NOT enabled!
2024-05-15 03:57:18,795 - root - INFO - DAC tile 0 DAC block 3 is NOT enabled!
2024-05-15 03:57:18,852 - root - INFO - DAC tile 1 DAC block 0 is enabled!
2024-05-15 03:57:18,854 - root - INFO - DAC tile 1 DAC block 1 is NOT enabled!
2024-05-15 03:57:18,857 - root - INFO - DAC tile 1 DAC block 2 is NOT enabled!
2024-05-15 03:57:18,859 - root - INFO - DAC tile 1 DAC block 3 is NOT enabled!
2024-05-1